# xVA Calculation Engine - Full Demo

This notebook demonstrates the complete xVA calculation workflow using our engine.

## Contents
1. Portfolio Definition
2. Monte Carlo Simulation
3. Exposure Calculation
4. Collateral Application
5. xVA Calculation
6. SA-CCR Regulatory Capital

In [ ]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# xVA Core imports
from xva_core import (
    FXForward,
    IRSwap,
    MonteCarloEngine,
    SACCRCalculator,
)
from xva_core.collateral import InitialMargin, VariationMargin
from xva_core.config.loader import create_default_market_config
from xva_core.exposure.metrics import ExposureMetrics
from xva_core.xva.result import XVAParams, calculate_all_xva

# Display settings
pd.set_option('display.float_format', '{:,.2f}'.format)
%matplotlib inline

## 1. Portfolio Definition

We define a sample portfolio with:
- 3 Interest Rate Swaps
- 2 FX Forwards

In [ ]:
# Define instruments
instruments = [
    # 5Y Payer Swap - $10M
    IRSwap(notional=10_000_000, fixed_rate=0.025, maturity=5.0, pay_fixed=True),

    # 3Y Receiver Swap - $15M
    IRSwap(notional=15_000_000, fixed_rate=0.020, maturity=3.0, pay_fixed=False),

    # 7Y Payer Swap - $8M
    IRSwap(notional=8_000_000, fixed_rate=0.030, maturity=7.0, pay_fixed=True),

    # 1Y FX Forward - Buy 5M EUR
    FXForward(notional_foreign=5_000_000, strike=1.12, maturity=1.0, buy_foreign=True),

    # 2Y FX Forward - Sell 3M EUR
    FXForward(notional_foreign=3_000_000, strike=1.08, maturity=2.0, buy_foreign=False),
]

total_notional = sum(inst.notional for inst in instruments)
print(f"Portfolio: {len(instruments)} trades")
print(f"Total notional: ${total_notional:,.0f}")

## 2. Monte Carlo Simulation

We simulate interest rates and FX using correlated OU and GBM processes.

In [ ]:
# Create market config and run simulation
market_config = create_default_market_config()

engine = MonteCarloEngine(
    n_paths=5000,
    horizon=7.0,
    dt=0.25,
    seed=42
)

result = engine.simulate(instruments, market_config)

print(f"Simulated {result.n_paths} paths over {result.n_steps} time steps")
print(f"Time grid: {result.time_grid[0]:.2f}Y to {result.time_grid[-1]:.2f}Y")

## 3. Exposure Calculation

In [ ]:
# Calculate exposure metrics
metrics = ExposureMetrics.from_mtm(result.mtm, result.time_grid)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(result.time_grid, metrics.epe / 1e6, 'r-', label='EPE', linewidth=2)
ax.plot(result.time_grid, metrics.ene / 1e6, 'g-', label='ENE', linewidth=2)
ax.plot(result.time_grid, metrics.pfe_95 / 1e6, 'm--', label='PFE 95%', linewidth=2)
ax.set_xlabel('Time (years)')
ax.set_ylabel('Exposure ($M)')
ax.set_title('Uncollateralized Exposure Profile')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

print(f"Peak EPE: ${metrics.peak_epe:,.0f}")
print(f"Peak ENE: ${metrics.peak_ene:,.0f}")

## 4. Collateral Application

Apply variation margin with:
- Threshold: $1M
- MTA: $100K
- MPR: 10 days

In [ ]:
# Apply VM
vm = VariationMargin(
    threshold=1_000_000,
    mta=100_000,
    mpr_days=10,
    days_per_step=91.25
)

collateral, coll_exposure = vm.apply(result.mtm, result.time_grid)

epe_coll = np.maximum(coll_exposure, 0).mean(axis=0)
ene_coll = np.maximum(-coll_exposure, 0).mean(axis=0)

# Calculate IM
im = InitialMargin(multiplier=1.5)
im_profile = im.calculate(coll_exposure)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(result.time_grid, metrics.epe / 1e6, 'r-', label='EPE (Uncoll)', linewidth=2)
axes[0].plot(result.time_grid, epe_coll / 1e6, 'r--', label='EPE (Coll)', linewidth=2)
axes[0].set_xlabel('Time (years)')
axes[0].set_ylabel('EPE ($M)')
axes[0].set_title('EPE: Collateral Impact')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].fill_between(result.time_grid, im_profile / 1e6, alpha=0.5, color='purple')
axes[1].plot(result.time_grid, im_profile / 1e6, 'purple', label='IM', linewidth=2)
axes[1].set_xlabel('Time (years)')
axes[1].set_ylabel('IM ($M)')
axes[1].set_title('Initial Margin Profile')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

reduction = (1 - epe_coll.max() / metrics.peak_epe) * 100
print(f"Collateral benefit: {reduction:.1f}%")
print(f"Average IM: ${im_profile.mean():,.0f}")

## 5. xVA Calculation

In [ ]:
# Get average discount factors
avg_df = result.df_domestic.mean(axis=0)

# Set xVA parameters
xva_params = XVAParams(
    lgd_counterparty=0.60,
    lgd_own=0.60,
    hazard_rate_counterparty=0.012,  # 120 bps
    hazard_rate_own=0.010,           # 100 bps
    funding_spread=0.01,             # 100 bps
    cost_of_capital=0.10,            # 10%
    capital_ratio=0.08,              # 8%
    im_multiplier=1.5,
)

# Calculate all xVA
xva_result = calculate_all_xva(
    epe=epe_coll,
    ene=ene_coll,
    discount_factors=avg_df,
    time_grid=result.time_grid,
    params=xva_params,
    im_profile=im_profile,
)

print(xva_result.summary(notional=total_notional))

In [ ]:
# Visualize xVA breakdown
metrics_names = ['CVA', 'DVA', 'FVA', 'MVA', 'KVA']
values = [xva_result.cva, -xva_result.dva, xva_result.fva, xva_result.mva, xva_result.kva]
colors = ['#FF4B4B', '#00CC96', '#FFA500', '#9467BD', '#1F77B4']

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(metrics_names, [v/1e6 for v in values], color=colors)
ax.set_ylabel('Value ($M)')
ax.set_title('xVA Components')
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)

# Add value labels
for bar, val in zip(bars, values, strict=False):
    height = bar.get_height()
    ax.annotate(f'${val/1e6:.2f}M',
                xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords='offset points',
                ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 6. SA-CCR Regulatory Capital

In [ ]:
# Calculate SA-CCR
saccr_calc = SACCRCalculator(alpha=1.4)
current_mtm = result.mtm[:, 0].mean()
saccr_result = saccr_calc.calculate(instruments, current_mtm=current_mtm)

print(saccr_result.summary())

# Comparison
avg_epe = epe_coll.mean()
print(f"\nInternal Avg EPE: ${avg_epe:,.0f}")
print(f"EAD/EPE Ratio: {saccr_result.ead/avg_epe:.2f}x")

In [ ]:
# Trade-level breakdown
trade_df = pd.DataFrame([
    {
        'Trade': ta.trade_id,
        'Asset Class': ta.asset_class.value,
        'Notional ($M)': ta.notional / 1e6,
        'Maturity (Y)': ta.maturity,
        'SF': ta.supervisory_factor,
        'Add-On ($M)': ta.addon / 1e6,
    }
    for ta in saccr_result.trade_addons
])

trade_df

## Summary

This demo showed the complete xVA calculation workflow:

1. **Portfolio**: 5 trades with ~$40M notional
2. **Simulation**: 5,000 Monte Carlo paths over 7 years
3. **Exposure**: EPE/ENE profiles with collateral benefit of ~50%
4. **xVA**: Total cost ~25-30 bps of notional
5. **SA-CCR**: Regulatory EAD ~2x average EPE